## Plots delays, throughputs, packet loss and path for a selected link

In [1]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
indices = "network_weather-2017.8.*"

my_query = {}

##  Select your link

In [2]:
# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"

## Get the data from Elasticsearch, and store the record based on its type

In [ ]:
my_query = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170801T000000Z', 'lt': '20170830T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer}},
                                {'term': {'src': srcSiteThroughputServer}},
                                {'term': {'src': destSiteOWDServer}},
                                {'term': {'src': destSiteThroughputServer}}
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer}},
                                {'term': {'dest': destSiteThroughputServer}},
                                {'term': {'dest': srcSiteOWDServer}},
                                {'term': {'dest': srcSiteThroughputServer}}
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll = list(scan(client=es, index=indices, query=my_query))


In [ ]:
site = {srcSiteOWDServer: sS, srcSiteThroughputServer: sS, destSiteOWDServer:dS, destSiteThroughputServer:dS}
data = {sS:pd.DataFrame(),dS:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
for res in scroll:
    if count<1: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site[res['_source']['src']]
    data[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'latency':
        data[s].set_value( ts, 'delay_median', res['_source']['delay_median'])
        data[s].set_value( ts, 'delay_mean', res['_source']['delay_mean'])
        data[s].set_value( ts, 'delay_sd', res['_source']['delay_sd'])
    elif column_type == 'packet_loss_rate':
        data[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    elif column_type == 'throughput':
        data[s].set_value( ts, 'throughput', res['_source']['throughput']/1024./1024./1024.)
    elif column_type == 'traceroute':
        data[s].set_value( ts, 'hash', res['_source']['hash'])
print(count)

{'_id': 'AV388P04NILXDji8UuZH', '_score': None, '_index': 'network_weather-2017.8.20', '_type': 'latency', 'sort': [6], '_source': {'srcProduction': True, 'srcSite': 'pic', 'delay_sd': 1.738300023650182, 'delay_mean': 15.604833333333334, 'dest': '128.142.223.247', 'destProduction': True, 'src': '193.109.172.188', 'destSite': 'CERN-PROD', 'srcVO': 'ATLAS', 'src_host': 'psl01.pic.es', 'delay_median': 15.43, 'timestamp': 1503187206000, 'ipv6': False, 'destVO': 'ATLAS', 'dest_host': 'perfsonar-lt.cern.ch', 'MA': '193.109.172.188'}}


In [ ]:
data[sS].sort_index(inplace=True) 
data[sS].describe()

data[dS].sort_index(inplace=True) 
data[dS].describe()

### Extract values

In [ ]:
F = data[sS]
B = data[dS]
Fthr = F[F.throughput.notnull()]    # Get the lines (i.e. times) whose throughput is not null
Bthr = B[B.throughput.notnull()]  
Fpl = F[F.packet_loss.notnull()]
Bpl = B[B.packet_loss.notnull()]
# Fd = F[F.delay_mean.notnull()]
# Bd = B[B.delay_mean.notnull()]
Fd = F[F.delay_mean<150]
Bd = B[B.delay_mean<150]
Jd = Fd.join(Bd,how='inner', rsuffix="_back")
Jd['dmean']= (Jd.delay_mean + Jd.delay_mean_back)/2

Fh = F[F.hash.notnull()]
Bh = B[B.hash.notnull()]
Fids=[];Bids=[]
Fu=[];Bu=[]

for fi in Fh.hash.tolist():
    if fi not in Fu: Fu.append(fi)
    Fids.append(Fu.index(fi))
for fi in Bh.hash.tolist():
    if fi not in Bu: Bu.append(fi)
    Bids.append(Bu.index(fi))

#### interpolate all the values and leave only onese where all 4 measurements are there

In [ ]:
NF=F.interpolate(method='index',limit=3,limit_direction='both')
NFall = NF[NF.throughput.notnull()]
NFall = NFall[NFall.delay_mean.notnull()]
NFall = NFall[NFall.packet_loss.notnull()]
NFall = NFall[NFall.hash.notnull()]

NB=B.interpolate(method='index',limit=3,limit_direction='both')
NBal = NB[NB.throughput.notnull()]
NBall = NBal[NBal.delay_mean.notnull()]
print(NFall.shape,NBall.shape)

In [ ]:
print(NFall)

In [ ]:
#NFall

In [ ]:
print( 'throughput  forward: ', Fthr.shape[0], ' backward: ', Bthr.shape[0])
print( 'packet loss forward: ', Fpl.shape[0],  ' backward: ', Bpl.shape[0])
print( 'delay       forward: ', Fd.shape[0],   ' backward: ', Bd.shape[0])
print( 'hash        forward: ', Fh.shape[0],   ' backward: ', Bh.shape[0])

In [ ]:
plt.figure(figsize=[16, 17])
gs = gridspec.GridSpec(4, 1)

ax0 = plt.subplot(gs[0])
ax0.plot(Fthr.timestamp.tolist(), Fthr.throughput.tolist(), ls='-', marker='>', c='r', label=sS+'->'+dS)
ax0.plot(Bthr.timestamp.tolist(), Bthr.throughput.tolist(), ls='-', marker='<', c='b', label=dS+'->'+sS)
ax0.set_xlabel('time')
ax0.set_ylabel('throughput [Gbps]')#,rotation='horizontal')
#ax0.xlim(xmin=0)
ax0.legend()

ax2 = plt.subplot(gs[1])
ax2.plot(Fpl.timestamp.tolist(), np.sqrt(Fpl.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax2.plot(Bpl.timestamp.tolist(), np.sqrt(Bpl.packet_loss).tolist(), ls='', marker='.', c='b', label=dS+'->'+sS)
ax2.set_xlabel('time')
ax2.set_ylabel('sqrt(packet loss) [%]')
ax2.legend()

ax4 = plt.subplot(gs[2])
ax4.plot(Fd.timestamp.tolist(), Fd.delay_mean.tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax4.plot(Bd.timestamp.tolist(), Bd.delay_mean.tolist(), ls='', marker='.', c='b', label=dS+'->'+sS)
ax4.plot(Jd.timestamp.tolist(), Jd.dmean.tolist(), ls='-', marker='.', c='g', label='mean delay')
ax4.set_xlabel('time')
ax4.set_ylabel('one way delay [ms]')
ax4.legend()

ax6 = plt.subplot(gs[3])
ax6.plot(Fh.timestamp.tolist(), Fids, ls='', marker='.', c='r', label=sS+'->'+dS)
ax6.plot(Bh.timestamp.tolist(), Bids, ls='', marker='.', c='b', label=dS+'->'+sS)
ax6.set_xlabel('time')
ax6.set_ylabel('path index')
ax6.legend()

#plt.savefig('Plots/4PlotForLink_' + sS +'_'+ dS + '.png',dpi=200)
plt.show()

In [ ]:
plt.figure(figsize=[8, 8])

#print (NFall.delay_mean.tolist(),NFall.throughput.tolist(),NFall.packet_loss.tolist())

cm = plt.cm.get_cmap('RdYlBu')

sc=plt.scatter(NFall.delay_mean.tolist(), NFall.packet_loss.tolist(), c=NFall.throughput.tolist(), vmin=0, vmax=10, cmap=cm, alpha=0.5, label=sS+'->'+dS)
#plt.scatter(NBall.delay_mean.tolist(), NBall.packet_loss.tolist(), c=NBall.throughput.tolist(), vmin=0, vmax=10, cmap=cm, alpha=0.5, label=dS+'->'+sS)
plt.xlabel('one way delay [ms]')
plt.ylabel('packet loss')#,rotation='horizontal')
plt.ylim(ymin=-0.0001)
plt.xscale("log")
plt.colorbar(sc)
plt.legend()

#ax2 = plt.subplot(gs[1])
#heatmap = ax2.pcolor(NFall.delay_mean.tolist(), NFall.throughput.tolist(), NFall.delay_mean.tolist(), cmap='PuBu_r')
# ax2.plot(Fpl.timestamp.tolist(), np.sqrt(Fpl.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
# ax2.plot(Bpl.timestamp.tolist(), np.sqrt(Bpl.packet_loss).tolist(), ls='', marker='.', c='b', label=dS+'->'+sS)
# ax2.set_xlabel('time')
# ax2.set_ylabel('sqrt(packet loss) [%]')
# ax2.legend()

# ax4 = plt.subplot(gs[2])
# ax4.plot(Fd.timestamp.tolist(), Fd.delay_mean.tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
# ax4.plot(Bd.timestamp.tolist(), Bd.delay_mean.tolist(), ls='', marker='.', c='b', label=dS+'->'+sS)
# ax4.plot(Jd.timestamp.tolist(), Jd.dmean.tolist(), ls='-', marker='.', c='g', label='mean delay')
# ax4.set_xlabel('time')
# ax4.set_ylabel('one way delay [ms]')
# ax4.legend()


#plt.savefig('Plots/CorrelationPlotsForLink_' + sS +'_'+ dS + '.png',dpi=200)
plt.show()

## learn SVR over 70% of data

In [ ]:
from sklearn import svm
trainingSetSize=int(NFall.shape[0]*.7)
XP = NFall[['delay_mean', 'packet_loss']].iloc[:trainingSetSize].values
XT = NFall[['delay_mean', 'packet_loss']].iloc[trainingSetSize:].values
y = np.ravel(NFall[[ 'throughput']].values.tolist()).tolist()
yP=y[:trainingSetSize]
yT=y[trainingSetSize:]
print('training set: ',XP.shape[0], 'testing set:', XT.shape[0])
print(len(yP),len(yT))

In [ ]:
clf = svm.SVR()
f = clf.fit(XP, yP)
print(f)

### predict for the rest of data

In [ ]:
tP=clf.predict(XT)
#TP

plt.figure(figsize=[5, 5])
plt.scatter(yT,tP)
plt.xlabel('actual values [Gbps]')
plt.ylabel('predicted values [Gbps]')

#plt.savefig('Plots/SVRpredictionResults' + sS +'_'+ dS + '.png',dpi=200)
plt.show()

In [ ]:
res = yT-tP
res.std()